In [188]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

In [189]:
# Define paths
RAW_DIR = Path("../input/raw/trip_records")
YELLOW_DIR = RAW_DIR / "yellow_taxi"
GREEN_DIR = RAW_DIR / "green_taxi"
HVFHV_DIR = RAW_DIR / "HVHFV"

# Read all parquet files from each folder and concatenate into single dataframes
df_yellow = pd.concat([pd.read_parquet(f) for f in YELLOW_DIR.glob("*.parquet")], ignore_index=True)
df_green = pd.concat([pd.read_parquet(f) for f in GREEN_DIR.glob("*.parquet")], ignore_index=True)
df_hvfhv = pd.concat([pd.read_parquet(f) for f in HVFHV_DIR.glob("*.parquet")], ignore_index=True)

print(f"Yellow taxi records: {len(df_yellow):,}")
print(f"Green taxi records: {len(df_green):,}")
print(f"HVFHV records: {len(df_hvfhv):,}")

Yellow taxi records: 13,389,587
Green taxi records: 221,006
HVFHV records: 78,608,184


In [190]:
# Standardize column names to lowercase for all dataframes
df_yellow.columns = df_yellow.columns.str.lower()
df_green.columns = df_green.columns.str.lower()
df_hvfhv.columns = df_hvfhv.columns.str.lower()

print("Column names standardized to lowercase")
print(f"Yellow columns: {list(df_yellow.columns)}")
print(f"Green columns: {list(df_green.columns)}")
print(f"HVFHV columns: {list(df_hvfhv.columns)}")

Column names standardized to lowercase
Yellow columns: ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']
Green columns: ['vendorid', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'ratecodeid', 'pulocationid', 'dolocationid', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']
HVFHV columns: ['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num', 'request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime', 'pulocationid', 'dolocationid', 'trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 's

# Data Cleaning

In [191]:
# Check data types and null values for all dataframes
def check_data_quality(df, name):
    """Display data types and null value counts for a dataframe."""
    print(f"{'='*60}")
    print(f"{name} - Shape: {df.shape}")
    print(f"{'='*60}")
    
    info_df = pd.DataFrame({
        'dtype': df.dtypes,
        'null_count': df.isnull().sum(),
        'null_pct': (df.isnull().sum() / len(df) * 100).round(2)
    })
    print(info_df)
    print()

check_data_quality(df_yellow, "Yellow Taxi")
check_data_quality(df_green, "Green Taxi")
check_data_quality(df_hvfhv, "HVFHV")

Yellow Taxi - Shape: (13389587, 19)


                                dtype  null_count  null_pct
vendorid                        int32           0      0.00
tpep_pickup_datetime   datetime64[us]           0      0.00
tpep_dropoff_datetime  datetime64[us]           0      0.00
passenger_count               float64     1221622      9.12
trip_distance                 float64           0      0.00
ratecodeid                    float64     1221622      9.12
store_and_fwd_flag             object     1221622      9.12
pulocationid                    int32           0      0.00
dolocationid                    int32           0      0.00
payment_type                    int64           0      0.00
fare_amount                   float64           0      0.00
extra                         float64           0      0.00
mta_tax                       float64           0      0.00
tip_amount                    float64           0      0.00
tolls_amount                  float64           0      0.00
improvement_surcharge         float64   

**Finding:** None of the relevant columns for the analysis have null values

## Yellow Taxi Data Cleaning

In [192]:
# Yellow Taxi - Data type conversions based on data dictionary
# Kept columns:
# - tpep_pickup_datetime, tpep_dropoff_datetime: datetime
# - pulocationid, dolocationid, passenger_count: int
# - fare_amount, total_amount, trip_distance: float

# Keep only relevant columns
cols_to_keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pulocationid', 'dolocationid',
                'trip_distance', 'fare_amount', 'total_amount']
df_yellow = df_yellow[cols_to_keep]

# Convert datetime columns
datetime_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
for col in datetime_cols:
    df_yellow[col] = pd.to_datetime(df_yellow[col])

# Convert integer columns (using nullable Int64 for columns that may have nulls)
int_cols = ['pulocationid', 'dolocationid']
for col in int_cols:
    df_yellow[col] = df_yellow[col].astype('Int64')

# Wait time is only available for HVFHV; set to null for Yellow Taxi
# Use Float64 so aggregates stay numeric
if 'wait_time' not in df_yellow.columns:
    df_yellow['wait_time'] = pd.Series(pd.NA, index=df_yellow.index, dtype='Float64')

print("Yellow Taxi data types after conversion:")
print(df_yellow.dtypes)


Yellow Taxi data types after conversion:
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
pulocationid                      Int64
dolocationid                      Int64
trip_distance                   float64
fare_amount                     float64
total_amount                    float64
wait_time                       Float64
dtype: object


In [193]:
# Filter yellow taxi dataframe to keep only January, April, July, October (the sampled months)
valid_months = [1, 4, 7, 10]
rows_before = len(df_yellow)
df_yellow = df_yellow[df_yellow['tpep_pickup_datetime'].dt.month.isin(valid_months)]
rows_after = len(df_yellow)
rows_dropped = rows_before - rows_after

print(f"Yellow Taxi - Rows before: {rows_before:,}")
print(f"Yellow Taxi - Rows after: {rows_after:,}")
print(f"Yellow Taxi - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

Yellow Taxi - Rows before: 13,389,587
Yellow Taxi - Rows after: 13,389,480
Yellow Taxi - Rows dropped: 107 (0.00%)


In [194]:
def audit_numeric(df, col):
    """Audit a numeric column for anomalies."""
    print(f"\n{'='*40}")
    print(f"Auditing: {col}")
    print(f"{'='*40}")
    
    series = df[col]
    
    # Basic stats
    print(f"\nBasic Stats:")
    print(series.describe())
    
    # Check for suspicious values
    print(f"\nPotential Issues:")
    print(f"  Nulls: {series.isna().sum():,} ({series.isna().mean()*100:.2f}%)")
    print(f"  Zeros: {(series == 0).sum():,} ({(series == 0).mean()*100:.2f}%)")
    print(f"  Negatives: {(series < 0).sum():,} ({(series < 0).mean()*100:.2f}%)")
    
    # Outliers (using IQR)
    q1, q99 = series.quantile([0.01, 0.99])
    print(f"\n  1st percentile: {q1}")
    print(f"  99th percentile: {q99}")
    
    extreme_low = (series < q1).sum()
    extreme_high = (series > q99).sum()
    print(f"  Below 1st pctl: {extreme_low:,}")
    print(f"  Above 99th pctl: {extreme_high:,}")

audit_numeric(df_yellow, 'fare_amount')
audit_numeric(df_yellow, 'trip_distance')
audit_numeric(df_yellow, 'total_amount')


Auditing: fare_amount

Basic Stats:
count    1.338948e+07
mean     1.913892e+01
std      1.935871e+01
min     -2.261200e+03
25%      9.300000e+00
50%      1.350000e+01
75%      2.190000e+01
max      5.000000e+03
Name: fare_amount, dtype: float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 4,781 (0.04%)
  Negatives: 229,042 (1.71%)

  1st percentile: -7.9
  99th percentile: 80.0
  Below 1st pctl: 131,107
  Above 99th pctl: 132,193

Auditing: trip_distance

Basic Stats:
count    1.338948e+07
mean     4.837142e+00
std      4.096487e+02
min      0.000000e+00
25%      1.020000e+00
50%      1.760000e+00
75%      3.360000e+00
max      3.663430e+05
Name: trip_distance, dtype: float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 235,868 (1.76%)
  Negatives: 0 (0.00%)

  1st percentile: 0.0
  99th percentile: 20.13
  Below 1st pctl: 0
  Above 99th pctl: 133,652

Auditing: total_amount

Basic Stats:
count    1.338948e+07
mean     2.774446e+01
std      2.408909e+01
min     -2.265450e+03
25% 

**Findings:**
* Negative values and zeros in fare_amount and total_amount might be refunds or cancelled trips, having no context I will drop these rows
* The max value of trip distance is 366,343 miles which is roughly traveling the circumference of the earth 9 times, New York city spans about 35 miles so I will drop any rows that lie outside the 0.1 to 100 miles range. I will also drop the zeros given it would make no sense to include them in the analysis

In [195]:
rows_before = len(df_yellow)

df_yellow = df_yellow[
    (df_yellow['fare_amount'] > 0) &
    (df_yellow['total_amount'] > 0) &
    (df_yellow['trip_distance'] >= 0.1) &
    (df_yellow['trip_distance'] <= 100)
]

rows_after = len(df_yellow)
rows_dropped = rows_before - rows_after

print(f"Yellow Taxi - Rows before filtering: {rows_before:,}")
print(f"Yellow Taxi - Rows after filtering: {rows_after:,}")
print(f"Yellow Taxi - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

Yellow Taxi - Rows before filtering: 13,389,480
Yellow Taxi - Rows after filtering: 12,896,459
Yellow Taxi - Rows dropped: 493,021 (3.68%)


## Green Taxi Data Cleaning

In [196]:
# Green Taxi - Data type conversions based on data dictionary
# Kept columns:
# - lpep_pickup_datetime, lpep_dropoff_datetime: datetime
# - pulocationid, dolocationid, passenger_count: int
# - fare_amount, total_amount, trip_distance: float

# Keep only relevant columns
cols_to_keep = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pulocationid', 'dolocationid',
                'trip_distance', 'fare_amount', 'total_amount']
df_green = df_green[cols_to_keep]

# Convert datetime columns
datetime_cols = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
for col in datetime_cols:
    df_green[col] = pd.to_datetime(df_green[col])

# Convert integer columns (using nullable Int64 for columns that may have nulls)
int_cols = ['pulocationid', 'dolocationid']
for col in int_cols:
    df_green[col] = df_green[col].astype('Int64')

# Wait time is only available for HVFHV; set to null for Green Taxi
# Use Float64 so aggregates stay numeric
if 'wait_time' not in df_green.columns:
    df_green['wait_time'] = pd.Series(pd.NA, index=df_green.index, dtype='Float64')

print("Green Taxi data types after conversion:")
print(df_green.dtypes)


Green Taxi data types after conversion:
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
pulocationid                      Int64
dolocationid                      Int64
trip_distance                   float64
fare_amount                     float64
total_amount                    float64
wait_time                       Float64
dtype: object


In [197]:
# Filter green taxi dataframe to keep only January, April, July, October
valid_months = [1, 4, 7, 10]
rows_before = len(df_green)
df_green = df_green[df_green['lpep_pickup_datetime'].dt.month.isin(valid_months)]
rows_after = len(df_green)
rows_dropped = rows_before - rows_after

print(f"Green Taxi - Rows before: {rows_before:,}")
print(f"Green Taxi - Rows after: {rows_after:,}")
print(f"Green Taxi - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

Green Taxi - Rows before: 221,006
Green Taxi - Rows after: 220,967
Green Taxi - Rows dropped: 39 (0.02%)


In [198]:
audit_numeric(df_green, 'fare_amount')
audit_numeric(df_green, 'trip_distance')
audit_numeric(df_green, 'total_amount')


Auditing: fare_amount

Basic Stats:
count    220967.000000
mean         18.017807
std          16.993607
min        -450.000000
25%          10.000000
50%          13.500000
75%          20.500000
max        1422.600000
Name: fare_amount, dtype: float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 187 (0.08%)
  Negatives: 719 (0.33%)

  1st percentile: 3.0
  99th percentile: 77.9
  Below 1st pctl: 988
  Above 99th pctl: 2,203

Auditing: trip_distance

Basic Stats:
count    220967.000000
mean         16.692203
std         992.180284
min           0.000000
25%           1.130000
50%           1.860000
75%           3.230000
max      201421.680000
Name: trip_distance, dtype: float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 11,648 (5.27%)
  Negatives: 0 (0.00%)

  1st percentile: 0.0
  99th percentile: 16.0
  Below 1st pctl: 0
  Above 99th pctl: 2,207

Auditing: total_amount

Basic Stats:
count    220967.000000
mean         23.869384
std          19.033013
min        -451.000000
2

**Findings:** Same logic from the yellow taxi dataframe applies to the green taxi dataframe

In [199]:
rows_before = len(df_green)

df_green = df_green[
    (df_green['fare_amount'] > 0) &
    (df_green['total_amount'] > 0) &
    (df_green['trip_distance'] >= 0.1) &
    (df_green['trip_distance'] <= 100)
]

rows_after = len(df_green)
rows_dropped = rows_before - rows_after

print(f"Green Taxi - Rows before filtering: {rows_before:,}")
print(f"Green Taxi - Rows after filtering: {rows_after:,}")
print(f"Green Taxi - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

Green Taxi - Rows before filtering: 220,967
Green Taxi - Rows after filtering: 206,982
Green Taxi - Rows dropped: 13,985 (6.33%)


## HVFHV Data Cleaning

In [200]:
# HVFHV - Data type conversions based on data dictionary
# Kept columns:
# - hvfhs_license_num: category (HV0002=Juno, HV0003=Uber, HV0004=Via, HV0005=Lyft)
# - request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime: datetime
# - pulocationid, dolocationid, trip_time: int
# - trip_miles, base_passenger_fare, tips: float
# - shared_request_flag, wav_request_flag, wav_match_flag: category (Y/N)

# Keep only relevant columns
cols_to_keep = ['hvfhs_license_num', 'request_datetime', 'on_scene_datetime', 'pickup_datetime',
                'dropoff_datetime', 'pulocationid', 'dolocationid', 'trip_miles', 'trip_time',
                'base_passenger_fare', 'tips', 'shared_request_flag', 'wav_request_flag',
                'wav_match_flag']
df_hvfhv = df_hvfhv[cols_to_keep]

# Convert datetime columns
datetime_cols = ['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']
for col in datetime_cols:
    df_hvfhv[col] = pd.to_datetime(df_hvfhv[col])

# Convert integer columns
int_cols = ['pulocationid', 'dolocationid', 'trip_time']
for col in int_cols:
    df_hvfhv[col] = df_hvfhv[col].astype('Int64')

# Convert string/category columns
category_cols = ['hvfhs_license_num']
for col in category_cols:
    df_hvfhv[col] = df_hvfhv[col].astype('category')

# Convert Y/N flag columns to category
flag_cols = ['shared_request_flag', 'wav_request_flag', 'wav_match_flag']
for col in flag_cols:
    df_hvfhv[col] = df_hvfhv[col].astype('category')

# Wait time in minutes between request and pickup
# Will be null when pickup_datetime is missing
df_hvfhv['wait_time'] = (
    df_hvfhv['pickup_datetime'] - df_hvfhv['request_datetime']
).dt.total_seconds() / 60

print("HVFHV data types after conversion:")
print(df_hvfhv.dtypes)


HVFHV data types after conversion:
hvfhs_license_num            category
request_datetime       datetime64[us]
on_scene_datetime      datetime64[us]
pickup_datetime        datetime64[us]
dropoff_datetime       datetime64[us]
pulocationid                    Int64
dolocationid                    Int64
trip_miles                    float64
trip_time                       Int64
base_passenger_fare           float64
tips                          float64
shared_request_flag          category
wav_request_flag             category
wav_match_flag               category
wait_time                     float64
dtype: object


In [201]:
# Filter HVFHV dataframe to keep only January, April, July, October
valid_months = [1, 4, 7, 10]
rows_before = len(df_hvfhv)
df_hvfhv = df_hvfhv[df_hvfhv['pickup_datetime'].dt.month.isin(valid_months)]
rows_after = len(df_hvfhv)
rows_dropped = rows_before - rows_after

print(f"HVFHV - Rows before: {rows_before:,}")
print(f"HVFHV - Rows after: {rows_after:,}")
print(f"HVFHV - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

HVFHV - Rows before: 78,608,184
HVFHV - Rows after: 78,608,184
HVFHV - Rows dropped: 0 (0.00%)


In [202]:
audit_numeric(df_hvfhv, 'trip_miles')
audit_numeric(df_hvfhv, 'trip_time')
audit_numeric(df_hvfhv, 'base_passenger_fare')
audit_numeric(df_hvfhv, 'tips')
audit_numeric(df_hvfhv, 'wait_time')


Auditing: trip_miles

Basic Stats:
count    7.860818e+07
mean     5.056058e+00
std      5.870998e+00
min      0.000000e+00
25%      1.570000e+00
50%      3.000000e+00
75%      6.340000e+00
max      4.555200e+02
Name: trip_miles, dtype: float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 11,186 (0.01%)
  Negatives: 0 (0.00%)

  1st percentile: 0.481
  99th percentile: 27.14
  Below 1st pctl: 785,498
  Above 99th pctl: 786,026

Auditing: trip_time

Basic Stats:
count     78608184.0
mean     1183.277328
std       835.949352
min              0.0
25%            599.0
50%            966.0
75%           1525.0
max          52060.0
Name: trip_time, dtype: Float64

Potential Issues:
  Nulls: 0 (0.00%)
  Zeros: 8 (0.00%)
  Negatives: 0 (0.00%)

  1st percentile: 196.0
  99th percentile: 4115.0
  Below 1st pctl: 775,474
  Above 99th pctl: 785,470

Auditing: base_passenger_fare

Basic Stats:
count    7.860818e+07
mean     2.564873e+01
std      2.241229e+01
min     -4.309000e+01
25%      1.2320

**Findings:** 
* Limiting trip distance to 0.1 to 100 miles
* Dropping negative and zero values for base_passenger_fare
* The tips and trip_time columns look fine

In [203]:
rows_before = len(df_hvfhv)

df_hvfhv = df_hvfhv[
    (df_hvfhv['base_passenger_fare'] > 0) &
    (df_hvfhv['trip_miles'] >= 0.1) &
    (df_hvfhv['trip_miles'] <= 100) &
    (df_hvfhv['wait_time'] > 0)
]

rows_after = len(df_hvfhv)
rows_dropped = rows_before - rows_after

print(f"HVFHV - Rows before filtering: {rows_before:,}")
print(f"HVFHV - Rows after filtering: {rows_after:,}")
print(f"HVFHV - Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

HVFHV - Rows before filtering: 78,608,184
HVFHV - Rows after filtering: 77,800,930
HVFHV - Rows dropped: 807,254 (1.03%)


# Data Aggregation

In [204]:
def add_time_features(df, pickup_col, dropoff_col):
    """Add time-based features for aggregation."""
    
    df['pickup_hour'] = df[pickup_col].dt.hour # Truncating to hour
    df['day_of_week'] = df[pickup_col].dt.dayofweek
    df['month'] = df[pickup_col].dt.month
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    df['trip_minutes'] = (df[dropoff_col] - df[pickup_col]).dt.total_seconds() / 60
    
    return df

# Apply time features to each dataframe
df_yellow = add_time_features(df_yellow, 'tpep_pickup_datetime', 'tpep_dropoff_datetime')
df_green = add_time_features(df_green, 'lpep_pickup_datetime', 'lpep_dropoff_datetime')
df_hvfhv = add_time_features(df_hvfhv, 'pickup_datetime', 'dropoff_datetime')

print(f"Sample from df_yellow:")
df_yellow[['pickup_hour', 'day_of_week', 'month', 'is_weekend', 'trip_minutes']].head()

Sample from df_yellow:


,pickup_hour,day_of_week,month,is_weekend,trip_minutes
0,0,0,1,0,19.800000
1,0,0,1,0,6.600000
2,0,0,1,0,17.916667
3,0,0,1,0,8.300000
4,0,0,1,0,6.100000


In [205]:
def aggregate_to_zone_time(df, vehicle_type, fare_col='fare_amount', distance_col='trip_distance'):
    """Aggregate trip data to zone x hour x is_weekend level."""
    
    agg = df.groupby(['pulocationid', 'pickup_hour', 'is_weekend']).agg(
        trip_count=('pickup_hour', 'count'),
        avg_fare=(fare_col, 'mean'),
        median_fare=(fare_col, 'median'),
        avg_trip_distance=(distance_col, 'mean'),
        avg_trip_minutes=('trip_minutes', 'mean'),
        total_fare=(fare_col, 'sum'),
        total_miles=(distance_col, 'sum'),
        avg_wait_time=('wait_time', 'mean'),
        std_wait_time=('wait_time', 'std')
    ).reset_index()
    
    agg['fare_per_mile'] = agg['total_fare'] / agg['total_miles']
    agg['vehicle_type'] = vehicle_type
    
    return agg

# Aggregate each dataframe
agg_yellow = aggregate_to_zone_time(df_yellow, 'yellow', fare_col='fare_amount', distance_col='trip_distance')
agg_green = aggregate_to_zone_time(df_green, 'green', fare_col='fare_amount', distance_col='trip_distance')
agg_hvfhv = aggregate_to_zone_time(df_hvfhv, 'hvfhv', fare_col='base_passenger_fare', distance_col='trip_miles')

print(f"Yellow aggregated: {agg_yellow.shape}")
print(f"Green aggregated: {agg_green.shape}")
print(f"HVFHV aggregated: {agg_hvfhv.shape}")

agg_yellow.head()


Yellow aggregated: (11278, 14)
Green aggregated: (5017, 14)
HVFHV aggregated: (12444, 14)


,pulocationid,pickup_hour,is_weekend,trip_count,avg_fare,median_fare,avg_trip_distance,avg_trip_minutes,total_fare,total_miles,avg_wait_time,std_wait_time,fare_per_mile,vehicle_type
0,1,0,0,2,46.600000,46.60,9.40000,23.816667,93.2,18.80,<NA>,<NA>,4.957447,yellow
1,1,2,0,1,100.000000,100.00,1.06000,4.983333,100.0,1.06,<NA>,<NA>,94.339623,yellow
2,1,5,0,8,73.812500,57.95,14.02625,15.702083,590.5,112.21,<NA>,<NA>,5.262454,yellow
3,1,6,0,3,30.700000,11.40,7.36000,14.088889,92.1,22.08,<NA>,<NA>,4.171196,yellow
4,1,6,1,3,132.166667,119.00,8.53000,10.077778,396.5,25.59,<NA>,<NA>,15.494334,yellow


In [206]:
# Combine all aggregated dataframes into a single one
df_aggregated = pd.concat([agg_yellow, agg_green, agg_hvfhv], ignore_index=True)

print(f"Combined aggregated dataframe shape: {df_aggregated.shape}")
print(f"\nVehicle type distribution:")
print(df_aggregated['vehicle_type'].value_counts())

Combined aggregated dataframe shape: (28739, 14)

Vehicle type distribution:
vehicle_type
hvfhv     12444
yellow    11278
green      5017
Name: count, dtype: int64


C:\Users\Danie\AppData\Local\Temp\ipykernel_17276\1607680300.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_aggregated = pd.concat([agg_yellow, agg_green, agg_hvfhv], ignore_index=True)


In [207]:
# Create zone-level aggregation (group by pulocationid only)
def aggregate_to_zone(df, vehicle_type, fare_col='fare_amount', distance_col='trip_distance'):
    """Aggregate trip data to zone level only."""
    
    agg = df.groupby(['pulocationid']).agg(
        trip_count=('pickup_hour', 'count'),
        avg_fare=(fare_col, 'mean'),
        median_fare=(fare_col, 'median'),
        avg_trip_distance=(distance_col, 'mean'),
        avg_trip_minutes=('trip_minutes', 'mean'),
        total_fare=(fare_col, 'sum'),
        total_miles=(distance_col, 'sum'),
        avg_wait_time=('wait_time', 'mean'),
        std_wait_time=('wait_time', 'std')
    ).reset_index()
    
    agg['fare_per_mile'] = agg['total_fare'] / agg['total_miles']
    agg['vehicle_type'] = vehicle_type
    
    return agg

# Aggregate each dataframe at zone level
agg_zone_yellow = aggregate_to_zone(df_yellow, 'yellow', fare_col='fare_amount', distance_col='trip_distance')
agg_zone_green = aggregate_to_zone(df_green, 'green', fare_col='fare_amount', distance_col='trip_distance')
agg_zone_hvfhv = aggregate_to_zone(df_hvfhv, 'hvfhv', fare_col='base_passenger_fare', distance_col='trip_miles')

# Combine into df_aggregated_zone
df_aggregated_zone = pd.concat([agg_zone_yellow, agg_zone_green, agg_zone_hvfhv], ignore_index=True)

print(f"Zone-level aggregated dataframe shape: {df_aggregated_zone.shape}")
print(f"\nVehicle type distribution:")
print(df_aggregated_zone['vehicle_type'].value_counts())


Zone-level aggregated dataframe shape: (767, 12)

Vehicle type distribution:
vehicle_type
hvfhv     262
yellow    261
green     244
Name: count, dtype: int64


C:\Users\Danie\AppData\Local\Temp\ipykernel_17276\937381944.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_aggregated_zone = pd.concat([agg_zone_yellow, agg_zone_green, agg_zone_hvfhv], ignore_index=True)


## Joining taxi zone information

In [208]:
# Load taxi zone lookup and standardize column names to lowercase
ZONE_LOOKUP_PATH = Path("../input/raw/other/taxi_zone_lookup.csv")
df_zones = pd.read_csv(ZONE_LOOKUP_PATH)
df_zones.columns = df_zones.columns.str.lower()

print(f"Zone lookup shape: {df_zones.shape}")
print(f"Columns: {list(df_zones.columns)}")
df_zones.head()

Zone lookup shape: (265, 4)
Columns: ['locationid', 'borough', 'zone', 'service_zone']


,locationid,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [209]:
# Join zone information to df_aggregated (zone x hour x is_weekend level)
df_aggregated = df_aggregated.merge(
    df_zones[['locationid', 'borough', 'zone', 'service_zone']],
    left_on='pulocationid',
    right_on='locationid',
    how='left'
).drop(columns=['locationid'])

print(f"df_aggregated shape after join: {df_aggregated.shape}")
print(f"Columns: {list(df_aggregated.columns)}")
df_aggregated.head()

df_aggregated shape after join: (28739, 17)
Columns: ['pulocationid', 'pickup_hour', 'is_weekend', 'trip_count', 'avg_fare', 'median_fare', 'avg_trip_distance', 'avg_trip_minutes', 'total_fare', 'total_miles', 'avg_wait_time', 'std_wait_time', 'fare_per_mile', 'vehicle_type', 'borough', 'zone', 'service_zone']


,pulocationid,pickup_hour,is_weekend,trip_count,avg_fare,median_fare,avg_trip_distance,avg_trip_minutes,total_fare,total_miles,avg_wait_time,std_wait_time,fare_per_mile,vehicle_type,borough,zone,service_zone
0,1,0,0,2,46.600000,46.60,9.40000,23.816667,93.2,18.80,NaN,NaN,4.957447,yellow,EWR,Newark Airport,EWR
1,1,2,0,1,100.000000,100.00,1.06000,4.983333,100.0,1.06,NaN,NaN,94.339623,yellow,EWR,Newark Airport,EWR
2,1,5,0,8,73.812500,57.95,14.02625,15.702083,590.5,112.21,NaN,NaN,5.262454,yellow,EWR,Newark Airport,EWR
3,1,6,0,3,30.700000,11.40,7.36000,14.088889,92.1,22.08,NaN,NaN,4.171196,yellow,EWR,Newark Airport,EWR
4,1,6,1,3,132.166667,119.00,8.53000,10.077778,396.5,25.59,NaN,NaN,15.494334,yellow,EWR,Newark Airport,EWR


In [210]:
# Join zone information to df_aggregated_zone (zone level only)
df_aggregated_zone = df_aggregated_zone.merge(
    df_zones[['locationid', 'borough', 'zone', 'service_zone']],
    left_on='pulocationid',
    right_on='locationid',
    how='left'
).drop(columns=['locationid'])

print(f"df_aggregated_zone shape after join: {df_aggregated_zone.shape}")
print(f"Columns: {list(df_aggregated_zone.columns)}")
df_aggregated_zone.head()

df_aggregated_zone shape after join: (767, 15)
Columns: ['pulocationid', 'trip_count', 'avg_fare', 'median_fare', 'avg_trip_distance', 'avg_trip_minutes', 'total_fare', 'total_miles', 'avg_wait_time', 'std_wait_time', 'fare_per_mile', 'vehicle_type', 'borough', 'zone', 'service_zone']


,pulocationid,trip_count,avg_fare,median_fare,avg_trip_distance,avg_trip_minutes,total_fare,total_miles,avg_wait_time,std_wait_time,fare_per_mile,vehicle_type,borough,zone,service_zone
0,1,153,83.239804,90.00,9.177516,15.027560,12735.69,1404.16,NaN,NaN,9.069971,yellow,EWR,Newark Airport,EWR
1,2,14,51.946429,53.75,14.197143,30.564286,727.25,198.76,NaN,NaN,3.658935,yellow,Queens,Jamaica Bay,Boro Zone
2,3,500,35.980780,36.50,9.242260,43.164167,17990.39,4621.13,NaN,NaN,3.893072,yellow,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,20292,17.499698,15.60,2.928168,16.181122,355103.87,59418.39,NaN,NaN,5.976329,yellow,Manhattan,Alphabet City,Yellow Zone
4,6,165,11.293273,0.01,7.442242,27.574747,1863.39,1227.97,NaN,NaN,1.517456,yellow,Staten Island,Arrochar/Fort Wadsworth,Boro Zone


## Joining U.S Census Bureau Information Via Area-Weighted Spatial Join

In [211]:
# Load census data fetched from src/fetch_census_data.py script
CENSUS_PATH = Path("../input/raw/census/nyc_census_tracts.parquet")
df_census = pd.read_parquet(CENSUS_PATH, engine='pyarrow')

print(f"Census data shape: {df_census.shape}")
print(f"Columns: {list(df_census.columns)}")
df_census.head()

Census data shape: (2327, 67)
Columns: ['total_population', 'population_below_poverty', 'hh_income_under_10k', 'hh_income_10k_to_15k', 'hh_income_15k_to_20k', 'hh_income_20k_to_25k', 'hh_income_25k_to_30k', 'hh_income_30k_to_35k', 'hh_income_35k_to_40k', 'hh_income_40k_to_45k', 'hh_income_45k_to_50k', 'hh_income_50k_to_60k', 'hh_income_60k_to_75k', 'hh_income_75k_to_100k', 'hh_income_100k_to_125k', 'hh_income_125k_to_150k', 'hh_income_150k_to_200k', 'hh_income_200k_plus', 'total_households', 'households_no_vehicle', 'commuters_total', 'commute_car_truck_van', 'commute_drove_alone', 'commute_carpooled', 'commute_public_transit', 'commute_taxi', 'commute_motorcycle', 'commute_bicycle', 'commute_walked', 'commute_other', 'commute_work_from_home', 'travel_time_total', 'travel_time_under_5min', 'travel_time_5_to_9min', 'travel_time_10_to_14min', 'travel_time_15_to_19min', 'travel_time_20_to_24min', 'travel_time_25_to_29min', 'travel_time_30_to_34min', 'travel_time_35_to_39min', 'travel_time

,total_population,population_below_poverty,hh_income_under_10k,hh_income_10k_to_15k,hh_income_15k_to_20k,hh_income_20k_to_25k,hh_income_25k_to_30k,hh_income_30k_to_35k,hh_income_35k_to_40k,hh_income_40k_to_45k,...,race_black_alone,race_american_indian_alone,race_asian_alone,race_pacific_islander_alone,race_other_alone,race_two_or_more,ethnicity_hispanic_latino,population_with_disability,geoid,borough
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36061000100,Manhattan
1,2568.0,1337.0,128.0,126.0,43.0,49.0,19.0,20.0,41.0,58.0,...,359.0,0.0,486.0,0.0,0.0,65.0,1487.0,2521.0,36061000201,Manhattan
2,6934.0,1724.0,294.0,590.0,159.0,59.0,106.0,157.0,227.0,111.0,...,1012.0,15.0,1566.0,0.0,0.0,87.0,2506.0,6922.0,36061000202,Manhattan
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36061000500,Manhattan
4,10179.0,3660.0,653.0,902.0,418.0,309.0,208.0,149.0,390.0,483.0,...,681.0,0.0,5023.0,0.0,233.0,129.0,2978.0,9982.0,36061000600,Manhattan


### Why Area-Weighted Spatial Join?

Census data is collected at the **census tract level** (~2,300 tracts in NYC), while taxi trip data is aggregated at the **taxi zone level** (265 zones defined by the TLC). These geographic boundaries do not align, a single taxi zone may contain multiple census tracts, and a census tract may span multiple taxi zones.

To properly join census demographics to taxi zones, we will use an **area-weighted spatial join**:

1. **Load shapefiles** for both geographies (census tracts and taxi zones)
2. **Compute polygon intersections** using `geopandas.overlay()` to find where tracts and zones overlap
3. **Calculate area weights** for each intersection: `weight = intersection_area / tract_area`
4. **Distribute census counts proportionally** by multiplying each tract's counts by the overlap weight
5. **Aggregate to taxi zone level** by summing all weighted values per zone

**Why this works for count data:**
If a census tract with 1,000 people is 60% within Zone A and 40% within Zone B, we allocate ~600 people to Zone A and ~400 to Zone B. This assumes uniform population distribution within tracts - not perfect, but a reasonable approximation given the granularity of census tracts.

**Important:** This method only works for **count-based variables** (population, households, etc.) that can be summed. Median or average values (like median income) cannot be properly aggregated this way, which is why we use income bracket counts instead.

In [212]:
# Define paths to shapefiles
SHAPEFILE_DIR = Path("../input/raw/shapefiles")
TAXI_ZONES_SHP = SHAPEFILE_DIR / "taxi_zones" / "taxi_zones.shp"
CENSUS_TRACTS_SHP = SHAPEFILE_DIR / "tiger_census_tracts" / "tl_2025_36_tract.shp"

# Load taxi zones shapefile
gdf_taxi_zones = gpd.read_file(TAXI_ZONES_SHP)
print(f"Taxi zones shape: {gdf_taxi_zones.shape}")
print(f"Taxi zones CRS: {gdf_taxi_zones.crs}")
print(f"Taxi zones columns: {list(gdf_taxi_zones.columns)}")
gdf_taxi_zones.head()

Taxi zones shape: (263, 7)
Taxi zones CRS: EPSG:2263
Taxi zones columns: ['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough', 'geometry']


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144..."


In [213]:
# Load census tracts shapefile
gdf_census_tracts = gpd.read_file(CENSUS_TRACTS_SHP)
print(f"Census tracts shape: {gdf_census_tracts.shape}")
print(f"Census tracts CRS: {gdf_census_tracts.crs}")
print(f"Census tracts columns: {list(gdf_census_tracts.columns)}")
gdf_census_tracts.head()

Census tracts shape: (5411, 14)
Census tracts CRS: EPSG:4269
Census tracts columns: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,029,008400,36029008400,1400000US36029008400,84,Census Tract 84,G5020,S,10966624,3505091,+42.9713848,-078.9194986,"POLYGON ((-78.94456 42.98506, -78.94216 42.992..."
1,36,103,123600,36103123600,1400000US36103123600,1236,Census Tract 1236,G5020,S,2302367,1082191,+40.6608399,-073.4145754,"POLYGON ((-73.42559 40.65629, -73.42529 40.656..."
2,36,103,146001,36103146001,1400000US36103146001,1460.01,Census Tract 1460.01,G5020,S,2225464,0,+40.7703277,-073.2532537,"POLYGON ((-73.26159 40.76307, -73.2615 40.7636..."
3,36,103,190402,36103190402,1400000US36103190402,1904.02,Census Tract 1904.02,G5020,S,44073411,23956,+40.8468673,-072.6336641,"POLYGON ((-72.72668 40.8339, -72.72515 40.8387..."
4,36,103,158709,36103158709,1400000US36103158709,1587.09,Census Tract 1587.09,G5020,S,13099359,110761,+40.8517499,-072.9216255,"POLYGON ((-72.94716 40.8556, -72.94649 40.8576..."


In [214]:
# Filter census tracts to only NYC counties (5 boroughs)
# NYC county FIPS codes: 061 (Manhattan), 047 (Brooklyn), 081 (Queens), 005 (Bronx), 085 (Staten Island)
nyc_county_fips = ['061', '047', '081', '005', '085']
gdf_census_tracts = gdf_census_tracts[gdf_census_tracts['COUNTYFP'].isin(nyc_county_fips)]

print(f"NYC census tracts shape after filtering: {gdf_census_tracts.shape}")
print(f"Tracts per county:")
print(gdf_census_tracts['COUNTYFP'].value_counts())

NYC census tracts shape after filtering: (2327, 14)
Tracts per county:
COUNTYFP
047    805
081    725
005    361
061    310
085    126
Name: count, dtype: int64


In [215]:
# Ensure both GeoDataFrames use the same CRS (coordinate reference system)
# Reproject to a projected CRS suitable for area calculations (NAD83 / New York Long Island - EPSG:2263)
target_crs = "EPSG:2263"

gdf_taxi_zones = gdf_taxi_zones.to_crs(target_crs)
gdf_census_tracts = gdf_census_tracts.to_crs(target_crs)

print(f"Taxi zones CRS after reprojection: {gdf_taxi_zones.crs}")
print(f"Census tracts CRS after reprojection: {gdf_census_tracts.crs}")

Taxi zones CRS after reprojection: EPSG:2263
Census tracts CRS after reprojection: EPSG:2263


In [216]:
# Calculate the area of each census tract (before intersection)
gdf_census_tracts['tract_area'] = gdf_census_tracts.geometry.area

# Create GEOID to match census data format (state + county + tract)
gdf_census_tracts['geoid'] = '36' + gdf_census_tracts['COUNTYFP'] + gdf_census_tracts['TRACTCE']

print(f"Sample GEOIDs from shapefile: {gdf_census_tracts['geoid'].head().tolist()}")
print(f"Sample GEOIDs from census data: {df_census['geoid'].head().tolist()}")

Sample GEOIDs from shapefile: ['36085024402', '36085027705', '36085012806', '36047024400', '36047023000']
Sample GEOIDs from census data: ['36061000100', '36061000201', '36061000202', '36061000500', '36061000600']


In [217]:
# Merge census demographic data with census tract geometries
gdf_census_with_data = gdf_census_tracts.merge(
    df_census,
    on='geoid',
    how='inner'
)

print(f"Census tracts with data shape: {gdf_census_with_data.shape}")
print(f"Tracts in shapefile: {len(gdf_census_tracts)}")
print(f"Tracts in census data: {len(df_census)}")
print(f"Matched tracts: {len(gdf_census_with_data)}")

Census tracts with data shape: (2327, 82)
Tracts in shapefile: 2327
Tracts in census data: 2327
Matched tracts: 2327


In [218]:
# Prepare taxi zones for overlay - keep only necessary columns
gdf_taxi_zones_simple = gdf_taxi_zones[['LocationID', 'geometry']].copy()
gdf_taxi_zones_simple = gdf_taxi_zones_simple.rename(columns={'LocationID': 'locationid'})

# Ensure locationid is integer type
gdf_taxi_zones_simple['locationid'] = gdf_taxi_zones_simple['locationid'].astype(int)

print(f"Taxi zones prepared for overlay: {gdf_taxi_zones_simple.shape}")

Taxi zones prepared for overlay: (263, 2)


In [219]:
# Perform spatial overlay (intersection) between census tracts and taxi zones
# This creates polygons representing the intersection of each tract with each zone
gdf_intersection = gpd.overlay(gdf_census_with_data, gdf_taxi_zones_simple, how='intersection')

print(f"Intersection result shape: {gdf_intersection.shape}")
print(f"Sample of intersection:")
gdf_intersection[['geoid', 'locationid', 'total_population']].head(10)

Intersection result shape: (5219, 83)
Sample of intersection:


,geoid,locationid,total_population
0,36085024402,44,4909.0
1,36085024402,84,4909.0
2,36085027705,23,6094.0
3,36085027705,118,6094.0
4,36085012806,172,5308.0
5,36085012806,176,5308.0
6,36047024400,22,3697.0
7,36047024400,26,3697.0
8,36047023000,26,4887.0
9,36047023100,17,3546.0


In [220]:
# Calculate the area of each intersection polygon
gdf_intersection['intersection_area'] = gdf_intersection.geometry.area

# Calculate the weight: proportion of tract area in this intersection
gdf_intersection['area_weight'] = gdf_intersection['intersection_area'] / gdf_intersection['tract_area']

# Sanity check: weights should be between 0 and 1
print(f"Area weight statistics:")
print(gdf_intersection['area_weight'].describe())

Area weight statistics:
count    5.219000e+03
mean     4.237352e-01
std      4.821187e-01
min      3.115767e-12
25%      4.165894e-05
50%      3.456858e-03
75%      9.998805e-01
max      1.000000e+00
Name: area_weight, dtype: float64


In [221]:
# Define the census columns to aggregate (all count-based columns)
census_count_columns = [
    # Population
    'total_population', 'population_below_poverty',
    
    # Income brackets
    'hh_income_under_10k', 'hh_income_10k_to_15k', 'hh_income_15k_to_20k',
    'hh_income_20k_to_25k', 'hh_income_25k_to_30k', 'hh_income_30k_to_35k',
    'hh_income_35k_to_40k', 'hh_income_40k_to_45k', 'hh_income_45k_to_50k',
    'hh_income_50k_to_60k', 'hh_income_60k_to_75k', 'hh_income_75k_to_100k',
    'hh_income_100k_to_125k', 'hh_income_125k_to_150k', 'hh_income_150k_to_200k',
    'hh_income_200k_plus',
    
    # Households & vehicles
    'total_households', 'households_no_vehicle',
    
    # Commute mode
    'commuters_total', 'commute_car_truck_van', 'commute_drove_alone',
    'commute_carpooled', 'commute_public_transit', 'commute_taxi',
    'commute_motorcycle', 'commute_bicycle', 'commute_walked',
    'commute_other', 'commute_work_from_home',
    
    # Travel time
    'travel_time_total', 'travel_time_under_5min', 'travel_time_5_to_9min',
    'travel_time_10_to_14min', 'travel_time_15_to_19min', 'travel_time_20_to_24min',
    'travel_time_25_to_29min', 'travel_time_30_to_34min', 'travel_time_35_to_39min',
    'travel_time_40_to_44min', 'travel_time_45_to_59min', 'travel_time_60_to_89min',
    'travel_time_90min_plus',
    
    # Employment
    'pop_16_plus', 'in_labor_force', 'civilian_labor_force',
    'employed', 'unemployed', 'armed_forces', 'not_in_labor_force',
    
    # Race/Ethnicity
    'race_total', 'race_white_alone', 'race_black_alone',
    'race_american_indian_alone', 'race_asian_alone', 'race_pacific_islander_alone',
    'race_other_alone', 'race_two_or_more', 'ethnicity_hispanic_latino',
    
    # Disability
    'population_with_disability'
]

print(f"Number of census columns to aggregate: {len(census_count_columns)}")

Number of census columns to aggregate: 61


In [222]:
# Apply area weights to all census count columns
# This distributes counts proportionally based on geographic overlap
for col in census_count_columns:
    gdf_intersection[f'{col}_weighted'] = gdf_intersection[col] * gdf_intersection['area_weight']

print("Applied area weights to all census columns")
print(f"Sample weighted values for total_population:")
gdf_intersection[['geoid', 'locationid', 'total_population', 'area_weight', 'total_population_weighted']].head(10)

Applied area weights to all census columns
Sample weighted values for total_population:


,geoid,locationid,total_population,area_weight,total_population_weighted
0,36085024402,44,4909.0,0.457237,2244.575493
1,36085024402,84,4909.0,0.000172,0.843885
2,36085027705,23,6094.0,0.000004,0.026072
3,36085027705,118,6094.0,0.999996,6093.973901
4,36085012806,172,5308.0,0.000836,4.438543
5,36085012806,176,5308.0,0.676991,3593.470378
6,36047024400,22,3697.0,0.000033,0.123575
7,36047024400,26,3697.0,0.999967,3696.876425
8,36047023000,26,4887.0,1.000000,4887.000000
9,36047023100,17,3546.0,0.000012,0.043248


In [223]:
# Aggregate weighted census data to taxi zone level
# Sum all weighted values for each taxi zone
weighted_columns = [f'{col}_weighted' for col in census_count_columns]

df_census_by_zone = gdf_intersection.groupby('locationid')[weighted_columns].sum().reset_index()

# Rename columns back to original names (remove '_weighted' suffix)
df_census_by_zone.columns = ['locationid'] + census_count_columns

# Round to integers since these are count data
for col in census_count_columns:
    df_census_by_zone[col] = df_census_by_zone[col].round(0).astype(int)

print(f"Census data aggregated to taxi zone level: {df_census_by_zone.shape}")
print(f"Number of taxi zones with census data: {len(df_census_by_zone)}")
df_census_by_zone.head()

Census data aggregated to taxi zone level: (259, 62)
Number of taxi zones with census data: 259


,locationid,total_population,population_below_poverty,hh_income_under_10k,hh_income_10k_to_15k,hh_income_15k_to_20k,hh_income_20k_to_25k,hh_income_25k_to_30k,hh_income_30k_to_35k,hh_income_35k_to_40k,...,race_total,race_white_alone,race_black_alone,race_american_indian_alone,race_asian_alone,race_pacific_islander_alone,race_other_alone,race_two_or_more,ethnicity_hispanic_latino,population_with_disability
0,2,45,4,0,1,0,0,0,1,0,...,45,36,1,0,4,0,0,1,4,45
1,3,30030,3539,687,412,234,177,373,255,194,...,30030,4987,10104,55,3315,31,230,518,10790,28445
2,4,19705,4814,1070,944,516,567,345,207,274,...,19705,6385,2320,94,2958,5,75,833,7036,19687
3,5,26418,1254,199,110,165,100,129,272,182,...,26418,18430,302,7,3281,0,141,359,3898,26402
4,6,14356,1475,340,141,182,155,85,144,202,...,14356,7980,706,0,3211,0,25,169,2265,14278


In [224]:
# Sanity check: Compare total population before and after spatial join
original_population = df_census['total_population'].sum()
aggregated_population = df_census_by_zone['total_population'].sum()

print(f"Original total population (census data): {original_population:,.0f}")
print(f"Aggregated total population (by zone): {aggregated_population:,.0f}")
print(f"Difference: {abs(original_population - aggregated_population):,.0f} ({abs(original_population - aggregated_population) / original_population * 100:.2f}%)")

Original total population (census data): 8,516,202
Aggregated total population (by zone): 8,080,887
Difference: 435,315 (5.11%)


**Note:** Small differences are expected due to:
  - Census tracts that don't overlap with any taxi zone (e.g., water areas)
  - Floating point precision in area calculations

In [225]:
# Join census data to df_aggregated (zone x hour x is_weekend level)
df_aggregated = df_aggregated.merge(
    df_census_by_zone,
    left_on='pulocationid',
    right_on='locationid',
    how='left'
).drop(columns=['locationid'])

print(f"df_aggregated shape after census join: {df_aggregated.shape}")
print(f"Columns: {list(df_aggregated.columns)}")
print(f"\nNull values in census columns (zones without census data):")
print(df_aggregated[census_count_columns].isnull().sum().sum())

df_aggregated shape after census join: (28739, 78)
Columns: ['pulocationid', 'pickup_hour', 'is_weekend', 'trip_count', 'avg_fare', 'median_fare', 'avg_trip_distance', 'avg_trip_minutes', 'total_fare', 'total_miles', 'avg_wait_time', 'std_wait_time', 'fare_per_mile', 'vehicle_type', 'borough', 'zone', 'service_zone', 'total_population', 'population_below_poverty', 'hh_income_under_10k', 'hh_income_10k_to_15k', 'hh_income_15k_to_20k', 'hh_income_20k_to_25k', 'hh_income_25k_to_30k', 'hh_income_30k_to_35k', 'hh_income_35k_to_40k', 'hh_income_40k_to_45k', 'hh_income_45k_to_50k', 'hh_income_50k_to_60k', 'hh_income_60k_to_75k', 'hh_income_75k_to_100k', 'hh_income_100k_to_125k', 'hh_income_125k_to_150k', 'hh_income_150k_to_200k', 'hh_income_200k_plus', 'total_households', 'households_no_vehicle', 'commuters_total', 'commute_car_truck_van', 'commute_drove_alone', 'commute_carpooled', 'commute_public_transit', 'commute_taxi', 'commute_motorcycle', 'commute_bicycle', 'commute_walked', 'commute_o

In [226]:
# Join census data to df_aggregated_zone (zone level only)
df_aggregated_zone = df_aggregated_zone.merge(
    df_census_by_zone,
    left_on='pulocationid',
    right_on='locationid',
    how='left'
).drop(columns=['locationid'])

print(f"df_aggregated_zone shape after census join: {df_aggregated_zone.shape}")
print(f"Columns: {list(df_aggregated_zone.columns)}")
print(f"\nNull values in census columns (zones without census data):")
print(df_aggregated_zone[census_count_columns].isnull().sum().sum())

df_aggregated_zone shape after census join: (767, 76)
Columns: ['pulocationid', 'trip_count', 'avg_fare', 'median_fare', 'avg_trip_distance', 'avg_trip_minutes', 'total_fare', 'total_miles', 'avg_wait_time', 'std_wait_time', 'fare_per_mile', 'vehicle_type', 'borough', 'zone', 'service_zone', 'total_population', 'population_below_poverty', 'hh_income_under_10k', 'hh_income_10k_to_15k', 'hh_income_15k_to_20k', 'hh_income_20k_to_25k', 'hh_income_25k_to_30k', 'hh_income_30k_to_35k', 'hh_income_35k_to_40k', 'hh_income_40k_to_45k', 'hh_income_45k_to_50k', 'hh_income_50k_to_60k', 'hh_income_60k_to_75k', 'hh_income_75k_to_100k', 'hh_income_100k_to_125k', 'hh_income_125k_to_150k', 'hh_income_150k_to_200k', 'hh_income_200k_plus', 'total_households', 'households_no_vehicle', 'commuters_total', 'commute_car_truck_van', 'commute_drove_alone', 'commute_carpooled', 'commute_public_transit', 'commute_taxi', 'commute_motorcycle', 'commute_bicycle', 'commute_walked', 'commute_other', 'commute_work_from_

In [227]:
print("Sample of df_aggregated_zone with census data:")
df_aggregated_zone[['pulocationid', 'zone', 'trip_count', 'total_population', 
                    'total_households', 'commute_public_transit', 'commute_taxi']].head(10)

Sample of df_aggregated_zone with census data:


,pulocationid,zone,trip_count,total_population,total_households,commute_public_transit,commute_taxi
0,1,Newark Airport,153,NaN,NaN,NaN,NaN
1,2,Jamaica Bay,14,45.0,18.0,3.0,0.0
2,3,Allerton/Pelham Gardens,500,30030.0,9909.0,4646.0,53.0
3,4,Alphabet City,20292,19705.0,9341.0,4239.0,71.0
4,6,Arrochar/Fort Wadsworth,165,14356.0,5143.0,2233.0,0.0
5,7,Astoria,7730,76914.0,35711.0,25373.0,329.0
6,8,Astoria Park,64,1.0,0.0,0.0,0.0
7,9,Auburndale,224,20544.0,7363.0,2748.0,7.0
8,10,Baisley Park,4433,39289.0,11970.0,6445.0,267.0
9,11,Bath Beach,358,24895.0,8697.0,4565.0,80.0


In [228]:
# Create derived demographic and income metrics for aggregated datasets
def add_derived_metrics(df):
    df['trips_per_capita'] = df['trip_count'] / df['total_population']
    df['poverty_rate'] = df['population_below_poverty'] / df['total_population']
    df['pct_no_vehicle'] = df['households_no_vehicle'] / df['total_households']
    df['pct_minority'] = (
        df['race_black_alone'] + df['ethnicity_hispanic_latino']
    ) / df['race_total']
    df['pct_white'] = df['race_white_alone'] / df['race_total']
    df['pct_public_transit'] = df['commute_public_transit'] / df['commuters_total']
    df['pct_low_income'] = (
        df['hh_income_under_10k'] + df['hh_income_10k_to_15k'] +
        df['hh_income_15k_to_20k'] + df['hh_income_20k_to_25k']
    ) / df['total_households']
    df['pct_high_income'] = (
        df['hh_income_150k_to_200k'] + df['hh_income_200k_plus']
    ) / df['total_households']

for df in (df_aggregated, df_aggregated_zone):
    add_derived_metrics(df)


In [229]:
# Save aggregated dataframes to parquet
PROCESSED_DIR = Path("../input/processed")

# Save df_aggregated (zone x hour x is_weekend level)
df_aggregated.to_parquet(PROCESSED_DIR / "aggregated_data_time_zone.parquet", index=False)
print(f"Saved df_aggregated to {PROCESSED_DIR / 'aggregated_data_time_zone.parquet'}")

# Save df_aggregated_zone (zone level only)
df_aggregated_zone.to_parquet(PROCESSED_DIR / "aggregated_data_zone.parquet", index=False)
print(f"Saved df_aggregated_zone to {PROCESSED_DIR / 'aggregated_data_zone.parquet'}")

Saved df_aggregated to ..\input\processed\aggregated_data_time_zone.parquet
Saved df_aggregated_zone to ..\input\processed\aggregated_data_zone.parquet
